In [ ]:
import torch
from utils import *
from collections import defaultdict
import matplotlib.pyplot as plt
import time

from models.rendering import *
from models.nerf import *

import metrics

from datasets import dataset_dict
from datasets.llff import *

torch.backends.cudnn.benchmark = True

img_wh = (320, 240)

# dataset = dataset_dict['llff'] \
#           ('/home/ubuntu/data/nerf_example_data/nerf_llff_data/fern/', 'test_train', spheric_poses=False,
#            img_wh=img_wh)

# dataset = dataset_dict['llff_nocs'] \
#           ('data/scene_1',
#            img_wh=img_wh)
dataset = dataset_dict['pd_multi_ae']

print("dataset", dataset)

In [2]:
from torch.utils.data import DataLoader
from functools import partial
from datasets.pd_multi_ae import collate_lambda_train, collate_lambda_val
kwargs = {'root_dir': '/home/ubuntu/nerf_pl/data/PDStep/v2',
          'img_wh': tuple(img_wh),
          'white_back': True,
          'model_type': 'refnerf'}

train_dataset = dataset(split='val', **kwargs)

dataloader =  DataLoader(train_dataset,
                  shuffle=True,
                  num_workers=0,
                  batch_size=12,
                  #batch_size=1,
                  pin_memory=False,
                  collate_fn = partial(collate_lambda_train, model_type='refnerf', ray_batch_size=2048)
                  #collate_fn = partial(collate_lambda_val, model_type='refnerf')
                        )

In [3]:
import torch
import numpy as np
a = torch.tensor([[0.6, 0.0, 0.0, 0.0],
[0.0, 0.4, 0.0, 0.0],
[0.0, 0.0, 1.2, 0.0],
[0.0, 0.0, 0.0,-0.4]])
b = torch.nonzero(a)

print(b.shape)
c = np.nonzero(a.numpy())
c = np.transpose(np.asarray(c))
print(np.transpose(np.asarray(c)).shape)
print(np.nonzero(a.numpy()))

print(np.equal(b.numpy(), c))

torch.Size([4, 2])
(2, 4)
(array([0, 1, 2, 3]), array([0, 1, 2, 3]))
[[ True  True]
 [ True  True]
 [ True  True]
 [ True  True]]


In [4]:
print("len train dataset", len(train_dataset))
count = 0
for i,data in enumerate(dataloader):
    if i>0:
        break
#     imgs, rgbs, rays, rays_d, view_dirs, instance_masks, radii, multloss, normals = data

    for k,v in data.items():
        print(k,v.shape)
    
    #print("imgs, rgbs, rays", imgs.shape, rgbs.shape, rays.shape, instance_masks.shape, radii.shape)
#     for k,v in data.items():
#         print("k, v", k, v.shape)
#     count+=1
# print(count)
#     print("data_rgb", data['rgbs'].shape)

len train dataset 162
instance_dir midsize_sedan_04_emg_01_black
instance_dir midsize_muscle_01_police_unmarkeddefault
instance_dir suv_medium_01_body_white


/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:134: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((w,h), Image.LANCZOS)
/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:137: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  seg_mask = seg_mask.resize((w,h), Image.LANCZOS)
/home/ubuntu/anaconda3/envs/nerf_pl/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:134: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((w,h), Image.LANCZOS)
/home/ubuntu/nerf

instance_dir compact_sport_01_red
instance_dir compact_mini_03_blueMetallic
instance_dir midsize_sedan_04_emg_02_white
instance_dir midsize_sedan_02_body_blue
instance_dir suv_large_01_body_blue
instance_dir compact_hatchback_01_body_gray


/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:134: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((w,h), Image.LANCZOS)
/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:137: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  seg_mask = seg_mask.resize((w,h), Image.LANCZOS)
/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:134: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((w,h), Image.LANCZOS)
/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:137: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  seg_mask = seg_mask.resize((w,h), Image.LANCZOS)
/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:134: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-0

instance_dir suv_medium_01_body_brown
instance_dir compact_mini_03_white
instance_dir midsize_sedan_01_body_red
img torch.Size([3, 116, 261])
img torch.Size([3, 115, 254])
img torch.Size([3, 112, 255])
img torch.Size([3, 156, 214])
img torch.Size([3, 181, 154])
img torch.Size([3, 126, 246])
img torch.Size([3, 174, 159])
img torch.Size([3, 128, 273])
img torch.Size([3, 97, 211])
img torch.Size([3, 112, 209])
img torch.Size([3, 96, 104])
img torch.Size([3, 134, 263])
src_imgs torch.Size([12, 3, 128, 128])
rays_o torch.Size([12, 2048, 3])
rays_d torch.Size([12, 2048, 3])
viewdirs torch.Size([12, 2048, 3])
target torch.Size([12, 2048, 3])
radii torch.Size([12, 2048, 1])
multloss (12, 1)
normals (12, 2048, 3)
instance_mask torch.Size([12, 2048, 1])
instance_dir suv_medium_02_blue
instance_dir fullsize_sedan_03_blue


/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:134: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((w,h), Image.LANCZOS)
/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:137: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  seg_mask = seg_mask.resize((w,h), Image.LANCZOS)
/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:134: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((w,h), Image.LANCZOS)
/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:137: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  seg_mask = seg_mask.resize((w,h), Image.LANCZOS)
/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:134: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-0

instance_dir compact_sport_01_metallic
instance_dir midsize_sedan_03_body_silver
instance_dir compact_mini_01_body_green
instance_dir midsize_muscle_01_police_01_blackWhite
instance_dir midsize_hatchback_01_body_green
instance_dir pickup_compact_01_red
instance_dir compact_luxury_001_body_gunmetal


/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:134: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((w,h), Image.LANCZOS)
/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:137: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  seg_mask = seg_mask.resize((w,h), Image.LANCZOS)
/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:134: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((w,h), Image.LANCZOS)
/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:137: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  seg_mask = seg_mask.resize((w,h), Image.LANCZOS)
/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:134: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-0

instance_dir van_micro_01_silver
instance_dir fullsize_sedan_02_blue
instance_dir midsize_hatchback_01_body_white
img torch.Size([3, 156, 230])
img torch.Size([3, 104, 270])
img torch.Size([3, 77, 192])
img torch.Size([3, 80, 99])
img torch.Size([3, 105, 187])
img torch.Size([3, 155, 165])
img torch.Size([3, 111, 260])
img torch.Size([3, 185, 145])
img torch.Size([3, 171, 238])
img torch.Size([3, 119, 158])
img torch.Size([3, 101, 248])
img torch.Size([3, 173, 239])


/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:134: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((w,h), Image.LANCZOS)
/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:137: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  seg_mask = seg_mask.resize((w,h), Image.LANCZOS)
/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:134: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((w,h), Image.LANCZOS)
/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:137: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  seg_mask = seg_mask.resize((w,h), Image.LANCZOS)
/home/ubuntu/nerf_pl/datasets/pd_multi_ae.py:134: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-0

In [7]:

B, N, _ = data["rays_o"].shape

rays_o, rays_d, radii = data["rays_o"].reshape(-1,3), data["rays_d"].reshape(-1,3), data["radii"].reshape(-1,3)

print("rays_o", rays_o.shape)

rays_o torch.Size([24576, 3])


In [ ]:
from models.resnet_encoder import MultiHeadImgEncoder
encoder = MultiHeadImgEncoder()

latents = encoder(data["src_imgs"])

print("latents", latents["density"].shape)
print("latents", latents["color"].shape)

In [ ]:
import matplotlib.pyplot as plt

print(data["img_wh"])

w,h = data["img_wh"]
print(data["src_imgs"].shape)
plt.imshow(data["src_imgs"].permute(1,2,0).numpy())
plt.show()

plt.imshow(data["target"].reshape(h,w,3).numpy())
plt.show()
    

In [ ]:
embedding_xyz = Embedding(10)
embedding_dir = Embedding(4)

nerf_coarse = NeRF()
nerf_fine = NeRF()

# ckpt_path = 'ckpts_old/fern/epoch=29.ckpt'
ckpt_path = 'ckpts_old/lego/epoch=15.ckpt'

load_ckpt(nerf_coarse, ckpt_path, model_name='nerf_coarse')
load_ckpt(nerf_fine, ckpt_path, model_name='nerf_fine')

nerf_coarse.cuda().eval()
nerf_fine.cuda().eval();

In [ ]:
models = {'coarse': nerf_coarse, 'fine': nerf_fine}
embeddings = {'xyz': embedding_xyz, 'dir': embedding_dir}

N_samples = 64
N_importance = 64
use_disp = False
chunk = 1024*32*4

@torch.no_grad()
def f(rays):
    """Do batched inference on rays using chunk."""
    B = rays.shape[0]
    results = defaultdict(list)
    for i in range(0, B, chunk):
        rendered_ray_chunks = \
            render_rays(models,
                        embeddings,
                        rays[i:i+chunk],
                        N_samples,
                        use_disp,
                        0,
                        0,
                        N_importance,
                        chunk,
                        dataset.white_back,
                        test_time=True)

        for k, v in rendered_ray_chunks.items():
            results[k] += [v]

    for k, v in results.items():
        results[k] = torch.cat(v, 0)
    return results

In [ ]:
sample = dataset[0]
rays = sample['rays'].cuda()

t = time.time()
results = f(rays)
torch.cuda.synchronize()
print(time.time()-t)

In [ ]:
img_gt = sample['rgbs'].view(img_wh[1], img_wh[0], 3)
img_pred = results['rgb_fine'].view(img_wh[1], img_wh[0], 3).cpu().numpy()
alpha_pred = results['opacity_fine'].view(img_wh[1], img_wh[0]).cpu().numpy()
depth_pred = results['depth_fine'].view(img_wh[1], img_wh[0])

print('PSNR', metrics.psnr(img_gt, img_pred).item())

plt.subplots(figsize=(15, 8))
plt.tight_layout()
plt.subplot(221)
plt.title('GT')
plt.imshow(img_gt)
plt.subplot(222)
plt.title('pred')
plt.imshow(img_pred)
plt.subplot(223)
plt.title('depth')
plt.imshow(visualize_depth(depth_pred).permute(1,2,0))
plt.show()